# Title: Convert shapefile to geojson (data from ECY)

## We get the shorelines from the WA dept of ecology, in form of a shapefile referenced in EPSG:2286. We convert it to EPSG 4326, which is the longitude/latitude projection, and then we output it as a geojson file.

In [63]:
import geopandas as gpd
import json

In [112]:
import re
import os


In [113]:
import pyproj

In [114]:
from shapely.geometry import Point, LineString

In [ ]:

#this cell pulls out every shapefile in the directory and puts them into a list

directory_path = 'WA_ECY_historic_shorelines'

pattern = re.compile(r'.*\.shp$')

filenames=[]

for filename in os.listdir(directory_path):
    if pattern.match(filename):
        filenames.append(filename)
        #print(filename)



In [109]:
#the output is a list of all the .shp filenames in the directory
print(filenames)


['shore_1997.shp', 'shore_196x.shp', 'shore_2015wa_2016or.shp', 'shore_1942.shp', 'shore_1995.shp', 'shore_197x.shp', 'shore_2005or_2006wa.shp', 'shore_192x.shp', 'shore_2001.shp', 'shore_198x.shp', 'shore_1999.shp', 'shore_195x.shp']


In [65]:
# now let's read in the shapefiles
# we'll use a dictionary named dict_shp to store the shapefiles

dict_shp={}

#namelist=[]

for filename in filenames:
    read_name="WA_ECY_historic_shorelines/"+filename
    dict_name=filename.split("e_")[1][:-4]
    #namelist.append(dict_name)
    dict_shp[dict_name]=gpd.read_file(read_name)

In [110]:
#the keys of the dictionary
dict_shp.keys()


dict_keys(['1997', '196x', '2015wa_2016or', '1942', '1995', '197x', '2005or_2006wa', '192x', '2001', '198x', '1999', '195x'])

In [111]:
#dict_shp["2015wa_2016or"]

In [68]:
#this cell creates a function to convert the state plane coordinates to geographic coordinates


proj = pyproj.Transformer.from_crs(2286, 4326, always_xy=True)   # 2286 is the state plane coordinate system for Washington State south, 4326 is the WGS84 coordinate system



# Convert State Plane coordinates to geographic coordinates

def convert_to_latlon(easting, northing):
    longitude, latitude = proj.transform(easting, northing)
    return (longitude, latitude)


In [70]:
#this cell convert the state plane coordinates to geographic coordinates with the function created above
for name in dict_shp.keys():
    for i in range(len(dict_shp[name]["geometry"])):
        if dict_shp[name]["geometry"][i]==None:
            continue
        dict_shp[name]["geometry"][i]=LineString([convert_to_latlon(dict_shp[name]["geometry"][i].coords[j][0],dict_shp[name]["geometry"][i].coords[j][1]) for j in range(len(dict_shp[name]["geometry"][i].coords))])

In [105]:
#after the conversion from EPSG:2286 to EPSG:4326, this cell converts the shp file to geojson file, and write the geojson file to the folder ECY_historical_shorelines
for entry in dict_shp.keys():
    print(entry)
    gjf = dict_shp[entry].to_json()
    print(type(gjf))
    gjf=json.loads(gjf)
    print(type(gjf))
    with open("ECY_historical_shorelines/"+"shore_"+entry+".geojson", "w") as f:
        json.dump(gjf, f)

1997
<class 'str'>
<class 'dict'>
196x
<class 'str'>
<class 'dict'>
2015wa_2016or
<class 'str'>
<class 'dict'>
1942
<class 'str'>
<class 'dict'>
1995
<class 'str'>
<class 'dict'>
197x
<class 'str'>
<class 'dict'>
2005or_2006wa
<class 'str'>
<class 'dict'>
192x
<class 'str'>
<class 'dict'>
2001
<class 'str'>
<class 'dict'>
198x
<class 'str'>
<class 'dict'>
1999
<class 'str'>
<class 'dict'>
195x
<class 'str'>
<class 'dict'>


In [107]:
#test whether we can open the geojson file we just created
with open("ECY_historical_shorelines/shore_192x.geojson", "r") as f:
    gjf=json.load(f)
    print(type(gjf))

len(gjf["features"])

<class 'dict'>


60

In [104]:
"""import json

json_string = '{"name": "John", "age": 30, "city": "New York"}'
dictionary = json.loads(json_string)

print(type(json_string),type(dictionary))"""


<class 'str'> <class 'dict'>
